In [2]:
import joblib
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from langchain.schema import Document

c:\Users\jilp1\AppData\Local\pypoetry\Cache\virtualenvs\teddynote-parser2-2bulC10K-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chunks = joblib.load('../../parser+splitter/data/20241025_company_622898000_splits.pkl')

In [4]:
# Elasticsearch 클라이언트 설정
es = Elasticsearch("http://localhost:9200")

In [5]:
# 1. 텍스트 임베딩 모델 로드 (Hugging Face SentenceTransformer)
model = SentenceTransformer('all-MiniLM-L6-v2')  # 예제 모델

In [6]:
# 2. 텍스트 데이터를 벡터로 변환
text = "Elasticsearch is a distributed, RESTful search engine."
vector = model.encode(text).tolist()  # 벡터를 리스트로 변환


In [7]:
# 3. Elasticsearch 인덱스 생성 (dense_vector 필드 포함)
index_name = "text_vectors"
if not es.indices.exists(index=index_name):
    es.indices.create(
        index=index_name,
        body={
            "mappings": {
                "properties": {
                    "text": {"type": "text"},
                    "embedding": {
                        "type": "dense_vector",
                        "dims": len(vector)  # 벡터 차원 수
                    }
                }
            }
        }
    )


In [12]:

for chunk in chunks:
    text = chunk.page_content
    vector = model.encode(text).tolist()  # 벡터를 리스트로 변환

    # 4. Elasticsearch에 데이터 삽입
    doc = {
        "text": text,
        "embedding": vector
    }
    response = es.index(index=index_name, body=doc)
    print("Document indexed:", response)


Document indexed: {'_index': 'text_vectors', '_id': 'TLKmbZQBPpm9ZCI6Zwu7', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 2}
Document indexed: {'_index': 'text_vectors', '_id': 'TbKmbZQBPpm9ZCI6aAsr', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 2}
Document indexed: {'_index': 'text_vectors', '_id': 'TrKmbZQBPpm9ZCI6aAuh', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 2}
Document indexed: {'_index': 'text_vectors', '_id': 'T7KmbZQBPpm9ZCI6aQsF', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 2}
Document indexed: {'_index': 'text_vectors', '_id': 'ULKmbZQBPpm9ZCI6aQtz', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 7, '_primary_term'

In [17]:

query = "재무 제표 중 투자의견이 낮을까 높을까"
# 5. Elasticsearch에서 벡터 검색 (예: 코사인 유사도)
query_vector = model.encode(query).tolist()
search_response = es.search(
    index=index_name,
    body={
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": """
                    cosineSimilarity(params.query_vector, 'embedding') + 1.0
                    """,
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
)

In [18]:
print("query vector", query_vector)

print("Search results:")
for hit in search_response['hits']['hits']:
    print(hit["_source"]["text"], "-> Score:", hit["_score"])


query vector [-0.005147207528352737, 0.034006472676992416, 0.03319163993000984, 0.020203102380037308, 0.00036415315116755664, -0.054268013685941696, 0.05603505298495293, 0.08251190185546875, -0.008205109275877476, -0.04620863124728203, 0.0668497234582901, 0.022474972531199455, 0.09629989415407181, 0.005464223213493824, 0.03367731720209122, -0.12692753970623016, 0.0512583889067173, 0.05612540990114212, -0.0938669815659523, -0.052288226783275604, 0.03662636876106262, -0.04566515237092972, 0.015062743797898293, 0.026172757148742676, -0.06164076179265976, 0.02874976396560669, -0.011155019514262676, 0.03061608597636223, 0.0886329784989357, 0.017939625307917595, -0.025695744901895523, 0.09190472960472107, 0.04022887721657753, -0.039081327617168427, -0.061789412051439285, 0.01029661949723959, -0.030587006360292435, -0.025106225162744522, -0.046981360763311386, 0.05642769858241081, -0.14470329880714417, -0.08269062638282776, 0.06674274057149887, -0.10989629477262497, 0.09537126868963242, -0.03

In [36]:
## top k개의 검색 결과 중에서 가장 유사한 문서를 찾아서 출력

query = "재무 제표 중 투자의견이 낮을까 높을까"
topk = 5
# 5. Elasticsearch에서 벡터 검색 (예: 코사인 유사도)
query_vector = model.encode(query).tolist()
search_response = es.search(
    index=index_name,
    size=topk,
    body={
        "query": {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": """
                    cosineSimilarity(params.query_vector, 'embedding') + 1.0
                    """,
                    "params": {"query_vector": query_vector}
                }
            }
        }
    }
)





C:\Users\jilp1\AppData\Local\Temp\ipykernel_28228\2110855131.py:7: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  search_response = es.search(


In [29]:
from pprint import pprint

In [37]:

print("query vector", query_vector)

print("Search results:")
for hit in search_response['hits']['hits']:
    print(f"ID: {hit['_id']}, Score: {hit['_score']} ----------------------------------------------------------------------------")
    print(hit["_source"]["text"])

query vector [-0.005147207528352737, 0.034006472676992416, 0.03319163993000984, 0.020203102380037308, 0.00036415315116755664, -0.054268013685941696, 0.05603505298495293, 0.08251190185546875, -0.008205109275877476, -0.04620863124728203, 0.0668497234582901, 0.022474972531199455, 0.09629989415407181, 0.005464223213493824, 0.03367731720209122, -0.12692753970623016, 0.0512583889067173, 0.05612540990114212, -0.0938669815659523, -0.052288226783275604, 0.03662636876106262, -0.04566515237092972, 0.015062743797898293, 0.026172757148742676, -0.06164076179265976, 0.02874976396560669, -0.011155019514262676, 0.03061608597636223, 0.0886329784989357, 0.017939625307917595, -0.025695744901895523, 0.09190472960472107, 0.04022887721657753, -0.039081327617168427, -0.061789412051439285, 0.01029661949723959, -0.030587006360292435, -0.025106225162744522, -0.046981360763311386, 0.05642769858241081, -0.14470329880714417, -0.08269062638282776, 0.06674274057149887, -0.10989629477262497, 0.09537126868963242, -0.03

In [12]:

## knn search

# 검색 쿼리
query = "재무 제표 중 투자의견이 낮을까 높을까"
topk = 5
query_vector = model.encode(query).tolist()

search_query = {
    "knn": {
        "field": "embedding",  # 벡터 필드 이름
        "query_vector": query_vector,
        "k": topk,
        "num_candidates": 1000  # 후보 문서 개수 (선택 사항, 기본값 1000)
    }
}

# 검색 요청
response = es.search(index=index_name, body=search_query)
print("Search results:")
for hit in response['hits']['hits']:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}, Source: {hit['_source']}")

Search results:
ID: XrKmbZQBPpm9ZCI6bgub, Score: 0.7407901, Source: {'text': '[ 산업 투자의견 ]\n\n당사는 산업에 대해 향후 1 년간 해당 업종의 수익률이 과거 수익률에 비해 양호한 흐름을 보일 것으로 예상되는 경우에 Positive(긍정 적) 의견을 제시하고 있습니다. 또한 향후 1 년간 수익률이 과거 수익률과 유사한 흐름을 보일 것으로 예상되는 경우에 Neutral(중립적) 의견을, 과거 수익률보다 부진한 흐름을 보일 것으로 예상되는 경우에 Negative(부정적) 의견을 제시하고 있습니다. 산업별 수익률 전 망은 해당 산업 내 분석대상 종목들에 대한 담당 애널리스트의 분석과 판단에 따릅니다.\n\n[ 당사 조사분석자료의 투자등급 부여 비중 ]\n\n(기준일: 2024 년 09 월 30 일)\n\n![](c:\\Users\\jilp1\\Documents\\boost ai\\Workspace\\code\\level4-nlp-finalproject-hackathon-nlp-13-lv3\\notebooks\\RAG\\parser+splitter\\data\\images\\table\\20241025_COMPANY_622898000_TABLE_Page_6_Index_69.png)\n\n| 투자등급 | 매수 | 중립 | 매도 | 합계 |\n| --- | --- | --- | --- | --- |\n| 금융투자상품의 비중 | 94.3% | 5.7% | 0.0% | 100.0% |', 'embedding': [0.00392674608156085, -0.009677763096988201, 0.025341516360640526, 0.003667088458314538, -0.006806171499192715, -0.07065946608781815, 0.02515607513487339, 0.07732729613780975, 0.0017804151866585016, -0.011742486618459225, 0.1

In [39]:
## bm25 + cosine similarity search


query_text = "재무 제표 중 투자의견이 낮을까 높을까"
query_vector = model.encode(query_text).tolist()

search_query = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
        # "match": {
        #   "text": query_text,
        # }
      },
      "script": {
        # BM25 점수는 _score 로 접근 가능
        # cosineSimilarity(params.query_vector, 'content_vector')는 벡터 유사도
        # alpha는 BM25와 벡터 유사도의 가중치
        "source": """
          double bm25Score = _score;
          double vectorSim = cosineSimilarity(params.query_vector, 'embedding') + 1.0;
          double alpha = 0.8; 
          
          return alpha * bm25Score + (1 - alpha) * vectorSim;
        """,
        "params": {
          "query_vector": query_vector
        }
      }
    }
  }
}

response = es.search(index=index_name, body=search_query)
print(response)
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")


{'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 22, 'relation': 'eq'}, 'max_score': 1.0965538, 'hits': [{'_index': 'text_vectors', '_id': 'XrKmbZQBPpm9ZCI6bgub', '_score': 1.0965538, '_source': {'text': '[ 산업 투자의견 ]\n\n당사는 산업에 대해 향후 1 년간 해당 업종의 수익률이 과거 수익률에 비해 양호한 흐름을 보일 것으로 예상되는 경우에 Positive(긍정 적) 의견을 제시하고 있습니다. 또한 향후 1 년간 수익률이 과거 수익률과 유사한 흐름을 보일 것으로 예상되는 경우에 Neutral(중립적) 의견을, 과거 수익률보다 부진한 흐름을 보일 것으로 예상되는 경우에 Negative(부정적) 의견을 제시하고 있습니다. 산업별 수익률 전 망은 해당 산업 내 분석대상 종목들에 대한 담당 애널리스트의 분석과 판단에 따릅니다.\n\n[ 당사 조사분석자료의 투자등급 부여 비중 ]\n\n(기준일: 2024 년 09 월 30 일)\n\n![](c:\\Users\\jilp1\\Documents\\boost ai\\Workspace\\code\\level4-nlp-finalproject-hackathon-nlp-13-lv3\\notebooks\\RAG\\parser+splitter\\data\\images\\table\\20241025_COMPANY_622898000_TABLE_Page_6_Index_69.png)\n\n| 투자등급 | 매수 | 중립 | 매도 | 합계 |\n| --- | --- | --- | --- | --- |\n| 금융투자상품의 비중 | 94.3% | 5.7% | 0.0% | 100.0% |', 'embedding': [0.003926746081560

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4, 'relation': 'eq'}, 'max_score': 2.9301028, 'hits': [{'_index': 'text_vectors', '_id': 'XLKmbZQBPpm9ZCI6bQvT', '_score': 2.9301028, '_source': {'text': '| 일 자 투자의견 목표주가 | 2016.08.12 투자등급변경 | 2022.10.27 Buy 110,000 | 2022.10.31 Buy 110,000 | 2022.11.16 Buy 110,000 | 2022.12.16 Buy 105,000 | 2023.01.12 Buy 105,000 |\n| --- | --- | --- | --- | --- | --- | --- |\n| 일 자 | 2023.02.02 | 2023.03.20 | 2023.04.04 | 2023.04.27 | 2023.05.16 | 2023.06.05 |\n| 투자의견 목표주가 | Buy 110,000 2023.06.21 | Buy 110,000 2023.07.27 | Buy 110,000 | Buy 110,000 2023.09.20 | Buy 110,000 2023.10.26 | Buy 110,000 |\n| 일 자 투자의견 목표주가 | Buy 150,000 | Buy 150,000 | 2023.09.14 Buy 150,000 | Buy 150,000 | Buy 150,000 | 2023.11.14 Buy 168,000 |\n| 일 자 투자의견 | 2023.11.29 Buy | 2023.12.14 | 2024.01.15 | 2024.01.26 | 2024.03.22 Buy | 2024.04.26 |\n|  |  |  |  |  |  |  |\n|  |  |  |  |  |  |  |\n|  |